## Introduction

## Shear Shallow Flow Method

## Shallow Moment Method

## Numerical Experiments

### TC 1: invisic 1d flow

### TC 2: dam-break with fingering

### TC 3: comparison with high-fidelity data

## Conclusion